# Part 3: Example-Based Explanations - David Lee Parnis

In [ ]:
!pip install pyale
!pip install alibi
!pip install dice-ml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline

# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import PartialDependenceDisplay
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from PyALE import ale
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# feed forward neural network for titanic

# load titanic dataset
titanic_train_df = pd.read_csv("/content/drive/MyDrive/ARI3205 - InterpretableAI_Project/Colab Notebooks/titanicDataset/train.csv")
# display completion message - done as an indicator is errors occur
if titanic_train_df is not None:
    print("----- <Titanic Training Dataset Loaded Successfully> -----")

titanic_test_df = pd.read_csv("/content/drive/MyDrive/ARI3205 - InterpretableAI_Project/Colab Notebooks/titanicDataset/test.csv")
# display completion message - done as an indicator is errors occur
if titanic_test_df is not None:
    print("----- <Titanic Testing Dataset Loaded Successfully> -----")

# display completion message - done as an indicator is errors occur

# Handle missing values
titanic_train_df['Age'].fillna(titanic_train_df['Age'].mean(), inplace=True)
titanic_train_df['Embarked'].fillna(titanic_train_df['Embarked'].mode()[0], inplace=True)

# Drop columns that are not needed for prediction
titanic_train_df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'], inplace=True)

# Convert categorical columns to numeric using one-hot encoding
titanic_train_df = pd.get_dummies(titanic_train_df, columns=['Sex', 'Embarked'], drop_first=True)

# Separate features and target
X_titanic = titanic_train_df.drop(columns=['Survived'])
y_titanic = titanic_train_df['Survived']

# split the dataset into training and testing sets with 80/20 split
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42)

# standardizing the features
scaler_titanic = StandardScaler()
X_train_titanic_scaled = scaler_titanic.fit_transform(X_train_titanic)
X_test_titanic_scaled = scaler_titanic.transform(X_test_titanic)

# define and train MLPClassifier model
mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=500,
    random_state=42
)

# train model
mlp_classifier.fit(X_train_titanic_scaled, y_train_titanic)

# get model predictions
predictions_titanic = mlp_classifier.predict(X_test_titanic_scaled)

# get accuracy score
accuracy_titanic = accuracy_score(y_test_titanic, predictions_titanic)

# display
print(f"Titanic Test Accuracy: {round((accuracy_titanic * 100), 2)}%")

----- <Titanic Training Dataset Loaded Successfully> -----
----- <Titanic Testing Dataset Loaded Successfully> -----
Titanic Test Accuracy: 80.45%


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
import dice_ml
from dice_ml import Dice
import pandas as pd

# Using DiCE to generate counterfactual explanations
# Prepare data for DiCE
categorical_features = ['Sex_male', 'Embarked_Q', 'Embarked_S']
continuous_features = [col for col in X_titanic.columns if col not in categorical_features]

data = dice_ml.Data(dataframe=pd.concat([X_train_titanic, y_train_titanic], axis=1),
                    continuous_features=continuous_features,
                    outcome_name='Survived',
                    categorical_features=categorical_features)

model = dice_ml.Model(model=mlp_classifier, backend='sklearn')

# Initialize Dice explainer
dice = Dice(data, model, method='random')

# Generate counterfactuals for an instance
cf_instance = pd.DataFrame([X_test_titanic.iloc[0]], columns=X_titanic.columns)  # Example instance

# Ensure that categorical values are properly formatted
def adjust_categorical_values(instance, categorical_features, data):
    for feature in categorical_features:
        if feature in instance.columns:
            instance_value = instance[feature].values[0]
            if instance_value not in data.data_df[feature].unique():
                instance[feature] = data.data_df[feature].mode()[0]
    return instance

cf_instance = adjust_categorical_values(cf_instance, categorical_features, data)

counterfactuals = dice.generate_counterfactuals(cf_instance, total_CFs=2, desired_class="opposite")

# Display results
counterfactuals.visualize_as_dataframe()


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  

Query instance (original outcome : 1)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Survived
0,3.0,29.699118,1.0,1.0,15.2458,1,0,1,1



Diverse Counterfactual set (new outcome: 0)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Survived
0,2.0,1.563705,8.0,3.0,15.2458,1,1,1,0
1,3.0,29.699118,1.0,5.0,0.3400,1,0,1,0


Counterfactual explanations are critical for debugging and decision-making:

Debugging Models: They reveal which features drive incorrect predictions and how small adjustments can flip outcomes, helping to identify biases or vulnerabilities in the model.
Actionable Insights: Counterfactuals provide actionable recommendations. For instance, if "age" and "fare" adjustments flip the survival prediction, it highlights their influence.
Fairness and Transparency: They help evaluate whether the model is making fair and interpretable decisions, ensuring ethical AI deployment.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

# Use KMeans for clustering
n_clusters = 5  # Define the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train_titanic_scaled)

# Get cluster centers
cluster_centers = kmeans.cluster_centers_

# Identify prototypes (closest points to cluster centers)
prototypes, _ = pairwise_distances_argmin_min(cluster_centers, X_train_titanic_scaled)

# Identify criticisms (points farthest from cluster centers)
distances = kmeans.transform(X_train_titanic_scaled)
criticisms = distances.max(axis=1).argsort()[-5:]  # Top 5 farthest points

# Display prototypes and criticisms
print("Prototype Indices:", prototypes)
print("Criticism Indices:", criticisms)

# Optionally, show the data points
print("\nPrototypes (representative points):")
print(X_train_titanic.iloc[prototypes])

print("\nCriticisms (outliers or poorly represented points):")
print(X_train_titanic.iloc[criticisms])


Prototype Indices: [461 345 149  84 305]
Criticism Indices: [175 476 367 517 370]

Prototypes (representative points):
     Pclass        Age  SibSp  Parch      Fare  Sex_male  Embarked_Q  \
325       1  36.000000      0      0  135.6333         0           0   
707       1  42.000000      0      0   26.2875         1           0   
436       3  21.000000      2      2   34.3750         0           0   
821       3  27.000000      0      0    8.6625         1           0   
46        3  29.699118      1      0   15.5000         1           1   

     Embarked_S  
325           0  
707           1  
436           1  
821           1  
46            0  

Criticisms (outliers or poorly represented points):
     Pclass   Age  SibSp  Parch      Fare  Sex_male  Embarked_Q  Embarked_S
678       3  43.0      1      6   46.9000         0           0           1
438       1  64.0      1      4  263.0000         1           0           1
737       1  35.0      0      0  512.3292         1        


Prototypes and criticisms are essential tools in interpretable AI, offering insights into the dataset and model behavior. Prototypes represent typical examples in the dataset, helping summarize its main structure and making the model’s decisions more relatable and understandable for stakeholders. They can also expose biases in the dataset if certain groups dominate. Criticisms, on the other hand, highlight outliers or edge cases that are poorly represented by the prototypes, revealing areas where the model may struggle or perform poorly. These outliers are crucial for identifying anomalies, improving model robustness, and addressing underrepresented groups in the dataset, which is especially important for ensuring fairness in high-stakes applications like healthcare or finance. Together, prototypes and criticisms provide a balanced view of both the dataset’s core structure and its limitations, enhancing transparency, trust, and actionable insights in AI systems.
